In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import findspark

findspark.init()

spark = SparkSession.builder \
        .appName("Practise") \
        .getOrCreate()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

#### Reading Data

In [ ]:
path = r"C:\Users\areeb\OneDrive\Documents\Desktop Backup\Personal Projects\Data Sets\laptop_prices.csv"

df = spark.read.format('csv') \
          .options(inferSchema = True, Header = True ) \
          .load(path)

#### 🧩 1. Basic EDA (Exploratory Data Analysis)

##### ✅ View basic info

In [ ]:
df.show(5)

+------+-----------+--------+---------+------------------+------------------+----------+--------------------+-----------+----------------+---------+
| Brand|  Processor|RAM (GB)|  Storage|               GPU|Screen Size (inch)|Resolution|Battery Life (hours)|Weight (kg)|Operating System|Price ($)|
+------+-----------+--------+---------+------------------+------------------+----------+--------------------+-----------+----------------+---------+
| Apple|AMD Ryzen 3|      64|512GB SSD|   Nvidia GTX 1650|              17.3| 2560x1440|                 8.9|       1.42|         FreeDOS|  3997.07|
| Razer|AMD Ryzen 7|       4|  1TB SSD|   Nvidia RTX 3080|              14.0|  1366x768|                 9.4|       2.57|           Linux|  1355.78|
|  Asus|   Intel i5|      32|  2TB SSD|   Nvidia RTX 3060|              13.3| 3840x2160|                 8.5|       1.74|         FreeDOS|  2673.07|
|Lenovo|   Intel i5|       4|256GB SSD|   Nvidia RTX 3080|              13.3|  1366x768|                10

In [ ]:
df.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Processor: string (nullable = true)
 |-- RAM (GB): integer (nullable = true)
 |-- Storage: string (nullable = true)
 |-- GPU: string (nullable = true)
 |-- Screen Size (inch): double (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Battery Life (hours): double (nullable = true)
 |-- Weight (kg): double (nullable = true)
 |-- Operating System: string (nullable = true)
 |-- Price ($): double (nullable = true)



In [ ]:
df.count()

11768

In [ ]:
df.columns

['Brand',
 'Processor',
 'RAM (GB)',
 'Storage',
 'GPU',
 'Screen Size (inch)',
 'Resolution',
 'Battery Life (hours)',
 'Weight (kg)',
 'Operating System',
 'Price ($)']

##### ✅ Summary statistics

In [ ]:
df.summary().show()

+-------+-------+-----------+------------------+---------+------------------+------------------+----------+--------------------+------------------+----------------+------------------+
|summary|  Brand|  Processor|          RAM (GB)|  Storage|               GPU|Screen Size (inch)|Resolution|Battery Life (hours)|       Weight (kg)|Operating System|         Price ($)|
+-------+-------+-----------+------------------+---------+------------------+------------------+----------+--------------------+------------------+----------------+------------------+
|  count|  11768|      11768|             11768|    11768|             11768|             11768|     11768|               11768|             11768|           11768|             11768|
|   mean|   NULL|       NULL|24.852821210061183|     NULL|              NULL|15.212304554724707|      NULL|   8.027855200543852|2.3411165873555464|            NULL|2183.5716077498296|
| stddev|   NULL|       NULL|21.762567369626534|     NULL|              NULL|1.4

In [ ]:
df.describe().show()

+-------+-------+-----------+------------------+---------+------------------+------------------+----------+--------------------+------------------+----------------+------------------+
|summary|  Brand|  Processor|          RAM (GB)|  Storage|               GPU|Screen Size (inch)|Resolution|Battery Life (hours)|       Weight (kg)|Operating System|         Price ($)|
+-------+-------+-----------+------------------+---------+------------------+------------------+----------+--------------------+------------------+----------------+------------------+
|  count|  11768|      11768|             11768|    11768|             11768|             11768|     11768|               11768|             11768|           11768|             11768|
|   mean|   NULL|       NULL|24.852821210061183|     NULL|              NULL|15.212304554724707|      NULL|   8.027855200543852|2.3411165873555464|            NULL|2183.5716077498296|
| stddev|   NULL|       NULL|21.762567369626534|     NULL|              NULL|1.4

#### ✅ Distinct values and counts

In [ ]:
df.select('Brand').distinct().show()

+---------+
|    Brand|
+---------+
|    Razer|
|       HP|
|     Dell|
|     Acer|
|     Asus|
|   Lenovo|
|  Samsung|
|Microsoft|
|    Apple|
|      MSI|
+---------+



In [ ]:
df.select('Brand').distinct().count()

10

#### ✅ Null and missing values

In [ ]:
df.select( [F.count(F.when(F.col(i).isNull(), i)).alias(i) for i in df.columns] ).show()

+-----+---------+--------+-------+---+------------------+----------+--------------------+-----------+----------------+---------+
|Brand|Processor|RAM (GB)|Storage|GPU|Screen Size (inch)|Resolution|Battery Life (hours)|Weight (kg)|Operating System|Price ($)|
+-----+---------+--------+-------+---+------------------+----------+--------------------+-----------+----------------+---------+
|    0|        0|       0|      0|  0|                 0|         0|                   0|          0|               0|        0|
+-----+---------+--------+-------+---+------------------+----------+--------------------+-----------+----------------+---------+



#### ✅ Value frequency

In [ ]:
df.groupBy(['Operating System', 'Brand']).agg( 
    F.count('*').alias('COUNT') ,
    F.round(sum('Price ($)')).alias( 'Total Price')
).orderBy('Operating System', ascending = True) \
.show()

+----------------+---------+-----+-----------+
|Operating System|    Brand|COUNT|Total Price|
+----------------+---------+-----+-----------+
|         FreeDOS|Microsoft|  287|   570055.0|
|         FreeDOS|      MSI|  319|   792907.0|
|         FreeDOS|     Dell|  273|   520356.0|
|         FreeDOS|   Lenovo|  262|   513508.0|
|         FreeDOS|       HP|  316|   648242.0|
|         FreeDOS|     Acer|  298|   573917.0|
|         FreeDOS|  Samsung|  299|   602430.0|
|         FreeDOS|     Asus|  292|   541066.0|
|         FreeDOS|    Razer|  275|   710756.0|
|         FreeDOS|    Apple|  331|   991495.0|
|           Linux|   Lenovo|  269|   533088.0|
|           Linux|     Dell|  301|   601901.0|
|           Linux|    Razer|  289|   841378.0|
|           Linux|    Apple|  332|   915533.0|
|           Linux|Microsoft|  299|   571669.0|
|           Linux|     Acer|  302|   569866.0|
|           Linux|  Samsung|  281|   552668.0|
|           Linux|     Asus|  294|   549803.0|
|           L

#### 🔄 2. Data Cleaning & Transformation

##### ✅ Column selection and renaming

In [ ]:
df.select('Brand', 'Processor').show(4)

+------+-----------+
| Brand|  Processor|
+------+-----------+
| Apple|AMD Ryzen 3|
| Razer|AMD Ryzen 7|
|  Asus|   Intel i5|
|Lenovo|   Intel i5|
+------+-----------+
only showing top 4 rows


In [ ]:
df = df.withColumnRenamed('Ram (GB)', 'RAM_GB')
df.show(3)

+-----+-----------+------+---------+---------------+------------------+----------+--------------------+-----------+----------------+---------+
|Brand|  Processor|RAM_GB|  Storage|            GPU|Screen Size (inch)|Resolution|Battery Life (hours)|Weight (kg)|Operating System|Price ($)|
+-----+-----------+------+---------+---------------+------------------+----------+--------------------+-----------+----------------+---------+
|Apple|AMD Ryzen 3|    64|512GB SSD|Nvidia GTX 1650|              17.3| 2560x1440|                 8.9|       1.42|         FreeDOS|  3997.07|
|Razer|AMD Ryzen 7|     4|  1TB SSD|Nvidia RTX 3080|              14.0|  1366x768|                 9.4|       2.57|           Linux|  1355.78|
| Asus|   Intel i5|    32|  2TB SSD|Nvidia RTX 3060|              13.3| 3840x2160|                 8.5|       1.74|         FreeDOS|  2673.07|
+-----+-----------+------+---------+---------------+------------------+----------+--------------------+-----------+----------------+---------+

#### ✅ Add new columns

In [ ]:
df.withColumn('Price_in_INR', F.col('Price ($)') * 83).show()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it